The following cell fixes the background color of tqdm background color in VSCode Jupyter Notebooks:

In [ ]:
%%html
<style>
.cell-output-ipywidget-background {
   background-color: transparent !important;
}
.jp-OutputArea-output {
   background-color: transparent;
}
</style>

Google Colab

In [1]:
# Датасет
!gdown 1wb6ayDuhhqOnFLjU4qWzeohiMnv7t8RK

Downloading...
From: https://drive.google.com/uc?id=1wb6ayDuhhqOnFLjU4qWzeohiMnv7t8RK
To: /content/train_dataset_train.csv
100% 13.6M/13.6M [00:00<00:00, 96.3MB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# !pip install git+https://github.com/huggingface/transformers.git
# !pip install -U sentence-transformers
# !pip install evaluate
# !pip install transformers[torch]
# !pip install demoji

---

In [1]:
import re
import json

import numpy as np
import pandas as pd

import demoji

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

import torch
from torch.utils.data import DataLoader, Dataset
import evaluate

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, T5TokenizerFast, pipeline

from tqdm.auto import tqdm

### Обработка датасета

In [2]:
data = pd.read_csv('train_dataset_train.csv', sep=';')
data.head()

,Исполнитель,Группа тем,Текст инцидента,Тема
0,Лысьвенский городской округ,Благоустройство,"'Добрый день. Сегодня, 20.08.22, моя мать шла ...",★ Ямы во дворах
1,Министерство социального развития ПК,Социальное обслуживание и защита,"'Пермь г, +79194692145. В Перми с ноября 2021 ...",Оказание гос. соц. помощи
2,Министерство социального развития ПК,Социальное обслуживание и защита,'Добрый день ! Скажите пожалуйста если подовал...,Дети и многодетные семьи
3,Город Пермь,Общественный транспорт,'Каждая из них не о чем. Люди на остановках хо...,Содержание остановок
4,Министерство здравоохранения,Здравоохранение/Медицина,'В Березниках у сына привитого откоронавируса ...,Технические проблемы с записью на прием к врачу


Убираем символ `'` в начале всех текстов инцидента

In [3]:
all([i[0] == "'" for i in data['Текст инцидента'].values])

True

In [4]:
data['Текст инцидента'] = data['Текст инцидента'].str.strip("'")
data['Текст инцидента'].head()

0    Добрый день. Сегодня, 20.08.22, моя мать шла п...
1    Пермь г, +79194692145. В Перми с ноября 2021 г...
2    Добрый день ! Скажите пожалуйста если подовала...
3    Каждая из них не о чем. Люди на остановках хот...
4    В Березниках у сына привитого откоронавируса з...
Name: Текст инцидента, dtype: object

Убираем тег `<br>` в начале всех текстов инцидента

In [5]:
data['Текст инцидента'].str.contains('<br>').sum()

3329

In [6]:
data['Текст инцидента'] = data['Текст инцидента'].str.replace('<br>', '\n')

Убираем ссылку на пользователя, которому адресован комментарий

In [7]:
data[data['Текст инцидента'].str.startswith("[")]['Текст инцидента']

22       [club185980418|Центр социальных выплат Пермско...
25       [club57433185|Пермь Первая], обратите внимание...
38       [id269738613|Дмитрий], в Краснокамске тоже ест...
59       [club80949945|Администрация города Лысьвы], ко...
64       [club201789187|ЦУР Пермского края] , здравству...
                               ...                        
23100    [club57433185|Пермь Первая], проблема с люком ...
23105    [id586879673|Жанна], Я дважды уже столкнулась ...
23113    [id153709709|Нина], ходить не возможно даже та...
23120    [club171874188|МАУ "СШ армейского рукопашного ...
23122    [club173907682|Березники официальные], а если ...
Name: Текст инцидента, Length: 1470, dtype: object

In [8]:
def remove_recipient(text):
    pattern = r"^\[[^\]]+\]"

    text_wo_recipient = re.sub(pattern, "", text).strip(', ')
    return text_wo_recipient

In [9]:
text = data[data['Текст инцидента'].str.startswith("[")]['Текст инцидента'].iloc[1]
print(text)

remove_recipient(text)

[club57433185|Пермь Первая], обратите внимание на организацию работы на ГЭС и на незаконченный ремонт дороги через переезд на ул. Писарева!


'обратите внимание на организацию работы на ГЭС и на незаконченный ремонт дороги через переезд на ул. Писарева!'

In [ ]:
# data['Текст инцидента'][data['Текст инцидента'].apply(remove_recipient).str.startswith("[")]

In [10]:
data['Текст инцидента'] = data['Текст инцидента'].apply(remove_recipient)

Если в тексте инцидента менее чем 4 слова, убираем такой текст

In [11]:
(data['Текст инцидента'].str.split().apply(len) < 4).sum()

638

In [12]:
data = data[data['Текст инцидента'].str.split().apply(len) >= 4]
data.head()

,Исполнитель,Группа тем,Текст инцидента,Тема
0,Лысьвенский городской округ,Благоустройство,"Добрый день. Сегодня, 20.08.22, моя мать шла п...",★ Ямы во дворах
1,Министерство социального развития ПК,Социальное обслуживание и защита,"Пермь г, +79194692145. В Перми с ноября 2021 г...",Оказание гос. соц. помощи
2,Министерство социального развития ПК,Социальное обслуживание и защита,Добрый день ! Скажите пожалуйста если подовала...,Дети и многодетные семьи
3,Город Пермь,Общественный транспорт,Каждая из них не о чем. Люди на остановках хот...,Содержание остановок
4,Министерство здравоохранения,Здравоохранение/Медицина,В Березниках у сына привитого откоронавируса з...,Технические проблемы с записью на прием к врачу


In [ ]:
# После того, как убрали ссылки, первые предложения могут начинаться
# с маленькой буквы
data['Текст инцидента'] = data['Текст инцидента'].apply(lambda text: text[0].upper() + text[1:])

Убираем эмоджи

In [14]:
data["Текст инцидента"] = data["Текст инцидента"].apply(lambda x: demoji.replace(x, ""))

### Spellchecker

In [ ]:
class SpellDataset(Dataset):
    def __init__(self, original_list):
        self.original_list = original_list

    def __len__(self):
        return len(self.original_list)

    def __getitem__(self, i):
        return 'Spell correct: ' + self.original_list[i]

spell_dataset = SpellDataset(data['Текст инцидента'].values)

In [17]:
spell_dataset[0]

'Spell correct: Добрый день. Сегодня, 20.08.22, моя мать шла по улице Ленина между домами 96 и 94. Фонари не горят, упала в яму, которую не видно. Сильно ударилась, остались синяки, очень больно. Благо шла не одна.\nУважаемая Администрация, сделайте с этим что-нибудь, да и не только с этим. Ходить опасно не только взрослым, но и детям. Если бы упал маленький ребёнок, было бы намного хуже. Фото прилагаю. Спасибо!'

In [22]:
spell_pipeline = pipeline(model='UrukHan/t5-russian-spell',
                          task='text2text-generation', batch_size=64, device='cuda')

In [ ]:
spells = []

for out in tqdm(spell_pipeline(spell_dataset), total=len(spell_dataset)):
    spells.append(out)

In [40]:
spells

['«Добрый день. Сегодня, 20.08.22, моя мать шла по улице Ленина между домами 96 и 94. Фонари не горят, упала в яму, которую не видно. Сильно ударилась, остались синяки, очень больно. Спасибо, шла не одна. Уважаемая Администрация, сделайте с этим что-нибудь. Да и не только с этим. Ходить опасно не только взрослым, но и детям. Если бы упал маленький ребёнок, было бы намного хуже. Фото прилагаю. Спасибо! Спасибо! Спасибо! Спасибо!! Спасибо!! Спасибо!!',
 'Каким образом можно получить льготу по проезду в такси в соц учреждения инвалиду 2 группы? Проезд в общественном транспорте не представляется.',
 'Здравствуйте! Скажите, пожалуйста, если подала на пособие с 3 до 7 декабря, когда можно повторно подать? . . . . Когда можно повторно подать? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . Вроде за 30 дней можно.',
 'А люди на остановках хотят укрыться от непогоды или слишком погоды. Присесть, поставить сумку. Лавочки на полторы попы? Отсутствие или намек на 

In [14]:
def check_spelling(input_text):
    try:
        encoded = spell_tokenizer(
            task_prefix + input_text,
            padding="longest",
            max_length=max_input,
            truncation=True,
            return_tensors="pt",
        ).to('cuda')

        predicts = spell_model.generate(**encoded)
        correct_text = spell_tokenizer.batch_decode(predicts, skip_special_tokens=True)[0]

        # Убираем лишние символы в начале предложения, если модель их добавила
        # correct_text = correct_text[correct_text.index(input_text[0]):]
        correct_text = correct_text.lstrip('.,[]«»')

        # Если модель выдает несколько одинаковых знаков препинания подряд, оставляем один
        correct_text = re.sub(r'([^\w\s])\1+', r'\1', correct_text)
    except:
        print(correct_text)

    return correct_text

In [26]:
# n = random.randint(0, data.shape[0])
n = 22
# print(n)

print(remove_recipient(data['Текст инцидента'][n]))
check_spelling(data['Текст инцидента'][n])

А какие выплаты для малоимущей(малообеспеченной) неполной семьи есть в вашем центре,весь доход семьи 10000 т.р. с небольшим


'А какие выплаты для малоимущей (малообеспеченной) неполной семьи есть в вашем центре? Весь доход семьи 10000 т. р. с небольшим.'

In [ ]:
data['Текст инцидента'] = data['Текст инцидента'].apply(check_spelling)

---

### Классификация

#### Обучение

In [18]:
data['Тема'].unique().shape

(195,)

In [19]:
id2label = {label: topic for label, topic in enumerate(data['Тема'].unique())}

label2id = {topic: label for label, topic in id2label.items()}

In [21]:
with open("id2label.json", 'w', encoding='utf-8') as f:
    json.dump(id2label, f, ensure_ascii=False, indent=4)

with open("label2id.json", 'w', encoding='utf-8') as f:
    json.dump(label2id, f, ensure_ascii=False, indent=4)

In [22]:
data['label'] = [label2id[topic] for topic in data['Тема']]
data

,Исполнитель,Группа тем,Текст инцидента,Тема,label
0,Лысьвенский городской округ,Благоустройство,"Добрый день. Сегодня, 20.08.22, моя мать шла п...",★ Ямы во дворах,0
1,Министерство социального развития ПК,Социальное обслуживание и защита,"Пермь г, +79194692145. В Перми с ноября 2021 г...",Оказание гос. соц. помощи,1
2,Министерство социального развития ПК,Социальное обслуживание и защита,Добрый день ! Скажите пожалуйста если подовала...,Дети и многодетные семьи,2
3,Город Пермь,Общественный транспорт,Каждая из них не о чем. Люди на остановках хот...,Содержание остановок,3
4,Министерство здравоохранения,Здравоохранение/Медицина,В Березниках у сына привитого откоронавируса з...,Технические проблемы с записью на прием к врачу,4
...,...,...,...,...,...
23122,Министерство социального развития ПК,Социальное обслуживание и защита,"А если ещё не погасили ипотеку, но площадь бол...",Улучшение жилищных условий,125
23123,Губахинский городской округ,ЖКХ,Город Гремячинск-— ситуация с теплом на улице ...,Ненадлежащее качество или отсутствие отопления,72
23124,Министерство здравоохранения,Здравоохранение/Медицина,"Здравствуйте у меня ребёнку 2 месяца , тест на...",Технические проблемы с записью на прием к врачу,4
23125,Лысьвенский городской округ,Благоустройство,А что творится с благоустройством дворов! Вооб...,Благоустройство придомовых территорий,122


In [23]:
# checkpoint = "xlm-roberta-base"
classification_checkpoint = "cointegrated/rubert-tiny2"

tokenizer = AutoTokenizer.from_pretrained(classification_checkpoint)

classification_model = AutoModelForSequenceClassification.from_pretrained(
    classification_checkpoint, num_labels = data['Тема'].unique().shape[0],
    id2label=id2label, label2id=label2id
)

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
class_weights = compute_class_weight("balanced", classes=list(label2id.keys()), y=data["Тема"])
class_weights = torch.tensor(class_weights, device=classification_model.device).to(torch.float)

In [25]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")

        outputs = classification_model(**inputs)
        logits = outputs.get("logits")

        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

In [ ]:
train_data, val_data = train_test_split(
    data[['Текст инцидента', 'label']], random_state=42, test_size=.1
)
val_data.head()

In [ ]:
class TextDataset(Dataset):
    def __init__(self, data_df, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.max_length = max_length

        self.sentences = data_df["Текст инцидента"].values
        self.labels = data_df['label'].values

    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self, i):
        sentence, label = self.sentences[i], self.labels[i]

        tokens = tokenizer(sentence, truncation="longest_first", padding="max_length", max_length=self.max_length)

        tokens['labels'] = label

        tokens = {key: torch.tensor(val).long() for key, val in tokens.items()}

        return tokens


train_dataset = TextDataset(train_data, tokenizer)
val_dataset = TextDataset(val_data, tokenizer)

train_dataset[0]

In [ ]:
accuracy = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    out = {}

    out.update(accuracy.compute(predictions=predictions, references=labels))
    out.update(f1_metric.compute(predictions=predictions, references=labels,
                                 average='weighted'))

    return out

In [ ]:
training_args = TrainingArguments(
    output_dir="promobot/models/rubert_tiny",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    # save_strategy="epoch",
    save_strategy='no',
    # load_best_model_at_end=True,
)

In [ ]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

#### Тест

In [ ]:
n = np.random.randint(val_data.shape[0])
n

In [ ]:
sentence = str(val_data["Текст инцидента"].iloc[n])

print(sentence, '\n', id2label[val_data["label"].iloc[n]])

tokens = tokenizer(sentence, truncation="longest_first", padding="max_length", max_length=512)

tokens = {key: torch.tensor(val).long() for key, val in tokens.items()}


for key in tokens:
    # tokens[key] = tokens[key].to("cuda").unsqueeze(0)
    tokens[key] = tokens[key].unsqueeze(0)

In [ ]:
pred = model(**tokens)
id2label[pred["logits"].argmax().item()]

#### Сохранение модели

In [ ]:
# torch.save(model, '/content/drive/MyDrive/models/promobot/rubert.pt')

#### Загрузка модели

---

In [104]:
data = pd.read_csv('data_corrected_spell_ner.csv')
data

,Исполнитель,Группа тем,Текст инцидента,Тема,Ners
0,Лысьвенский городской округ,Благоустройство,"Добрый день. Сегодня, 20 августа, моя мать шла...",★ Ямы во дворах,"[['LOC', 'Ленина']]"
1,Министерство социального развития ПК,Социальное обслуживание и защита,"Пермь, г. , +791692145. В Перми с ноября 2021 ...",Оказание гос. соц. помощи,"[['LOC', 'Пермь'], ['LOC', 'Перми']]"
2,Министерство социального развития ПК,Социальное обслуживание и защита,"Добрый день! Скажите, пожалуйста, если подала ...",Дети и многодетные семьи,[]
3,Город Пермь,Общественный транспорт,Каждая из них не о чем. Люди на остановках хот...,Содержание остановок,[]
4,Министерство здравоохранения,Здравоохранение/Медицина,"В Березниках у сына, привитого от коронавируса...",Технические проблемы с записью на прием к врачу,"[['LOC', 'Березниках']]"
...,...,...,...,...,...
22485,Министерство социального развития ПК,Социальное обслуживание и защита,"А если ещё не погасили ипотеку, но площадь бол...",Улучшение жилищных условий,[]
22486,Губахинский городской округ,ЖКХ,Город Гремячинск — ситуация с теплом на улице ...,Ненадлежащее качество или отсутствие отопления,"[['LOC', 'Гремячинск'], ['PER', 'Кожевникова']]"
22487,Министерство здравоохранения,Здравоохранение/Медицина,"Здравствуйте, у меня ребёнку 2 месяца. Тест на...",Технические проблемы с записью на прием к врачу,[]
22488,Лысьвенский городской округ,Благоустройство,А что творится с благоустройством дворов?!!! В...,Благоустройство придомовых территорий,"[['PER', 'Федосеев'], ['LOC', 'Оборина']]"


In [42]:
import re

In [105]:
def remove_extra_symbols(text):
    # Убираем лишние символы в начале предложения, если модель их добавила
    # correct_text = correct_text[correct_text.index(input_text[0]):]
    text = text.lstrip('.,[]«»')

    # Если модель выдает несколько одинаковых знаков препинания подряд, оставляем один
    text = re.sub(r'([^\w\s])\1+', r'\1', text)

    return text

data['Текст инцидента'] = data['Текст инцидента'].apply(remove_extra_symbols)
data.head()

,Исполнитель,Группа тем,Текст инцидента,Тема,Ners
0,Лысьвенский городской округ,Благоустройство,"Добрый день. Сегодня, 20 августа, моя мать шла...",★ Ямы во дворах,"[['LOC', 'Ленина']]"
1,Министерство социального развития ПК,Социальное обслуживание и защита,"Пермь, г. , +791692145. В Перми с ноября 2021 ...",Оказание гос. соц. помощи,"[['LOC', 'Пермь'], ['LOC', 'Перми']]"
2,Министерство социального развития ПК,Социальное обслуживание и защита,"Добрый день! Скажите, пожалуйста, если подала ...",Дети и многодетные семьи,[]
3,Город Пермь,Общественный транспорт,Каждая из них не о чем. Люди на остановках хот...,Содержание остановок,[]
4,Министерство здравоохранения,Здравоохранение/Медицина,"В Березниках у сына, привитого от коронавируса...",Технические проблемы с записью на прием к врачу,"[['LOC', 'Березниках']]"


In [106]:
def correct_ners(ners):
    ners = eval(ners)
    ners = ', '.join([f'{ner[0]}: {ner[1]}' for ner in ners if ner[0] in ['LOC', 'ORG']])
    return ners

data['Ners'] = data['Ners'].apply(correct_ners)
data.head()

,Исполнитель,Группа тем,Текст инцидента,Тема,Ners
0,Лысьвенский городской округ,Благоустройство,"Добрый день. Сегодня, 20 августа, моя мать шла...",★ Ямы во дворах,LOC: Ленина
1,Министерство социального развития ПК,Социальное обслуживание и защита,"Пермь, г. , +791692145. В Перми с ноября 2021 ...",Оказание гос. соц. помощи,"LOC: Пермь, LOC: Перми"
2,Министерство социального развития ПК,Социальное обслуживание и защита,"Добрый день! Скажите, пожалуйста, если подала ...",Дети и многодетные семьи,
3,Город Пермь,Общественный транспорт,Каждая из них не о чем. Люди на остановках хот...,Содержание остановок,
4,Министерство здравоохранения,Здравоохранение/Медицина,"В Березниках у сына, привитого от коронавируса...",Технические проблемы с записью на прием к врачу,LOC: Березниках


In [109]:
def data4contractor_prediction(row):
    full_text = f"{row['Текст инцидента']};\n{row['Тема']};\n{row['Группа тем']};"

    if row['Ners']:
        full_text += f"\n{row['Ners']}"

    return full_text

data['full_text_wo_contractor'] = data[['Текст инцидента', 'Тема', 'Группа тем', 'Ners']].apply(data4contractor_prediction, axis=1)
data.head()

,Исполнитель,Группа тем,Текст инцидента,Тема,Ners,full_text_wo_contractor
0,Лысьвенский городской округ,Благоустройство,"Добрый день. Сегодня, 20 августа, моя мать шла...",★ Ямы во дворах,LOC: Ленина,"Добрый день. Сегодня, 20 августа, моя мать шла..."
1,Министерство социального развития ПК,Социальное обслуживание и защита,"Пермь, г. , +791692145. В Перми с ноября 2021 ...",Оказание гос. соц. помощи,"LOC: Пермь, LOC: Перми","Пермь, г. , +791692145. В Перми с ноября 2021 ..."
2,Министерство социального развития ПК,Социальное обслуживание и защита,"Добрый день! Скажите, пожалуйста, если подала ...",Дети и многодетные семьи,,"Добрый день! Скажите, пожалуйста, если подала ..."
3,Город Пермь,Общественный транспорт,Каждая из них не о чем. Люди на остановках хот...,Содержание остановок,,Каждая из них не о чем. Люди на остановках хот...
4,Министерство здравоохранения,Здравоохранение/Медицина,"В Березниках у сына, привитого от коронавируса...",Технические проблемы с записью на прием к врачу,LOC: Березниках,"В Березниках у сына, привитого от коронавируса..."


In [113]:
print(data['full_text_wo_contractor'].iloc[1], data['full_text_wo_contractor'].iloc[3], sep='\n\n')

Пермь, г. , +791692145. В Перми с ноября 2021 года не работает социальное такси. Каким образом можно получить льготу по проезду в такси в соц учреждения инвалиду 2 группы? Проезд в общественном транспорте не возможен. Да и проездного льготного не представляется.;
Оказание гос. соц. помощи;
Социальное обслуживание и защита;
LOC: Пермь, LOC: Перми

Каждая из них не о чем. Люди на остановках хотят укрыться от непогоды или слишком погоды. Присесть, поставить сумку. Лавочки на полторы минуты? Отсутствие или намек на присутствие спинки? Конструкторы, понятно, что не ждут транспорт на остановках. А гаджеты заряжать дома надо. Интерактивные панели и близко не нужны. У каждого в руках телефоны - информации вагон. Разместить коды для быстрого доступа на ресурсы и сайты по транспорту хватит.;
Содержание остановок;
Общественный транспорт;


In [114]:
data.to_csv('data_corrected_spell_ner_full_text.csv', index=False)

---

In [2]:
import json
import pandas as pd
import plotly.express as px


big_topic_path = 'topic2big_topic.json'

with open(big_topic_path, 'r', encoding='UTF-8') as file:
    json_dict = json.load(file)

df = pd.DataFrame(list(json_dict.items()), columns=['Тема', 'Группа тем'])

fig = px.sunburst(df, path=['Группа тем', 'Тема'])
fig.show()